<a href="https://colab.research.google.com/github/Capou/FishClassifier/blob/master/fish_classifier_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
#!pip install fastbook

     |████████████████████████████████| 727kB 4.3MB/s 
     |████████████████████████████████| 194kB 43.7MB/s 
     |████████████████████████████████| 1.2MB 27.9MB/s 
     |████████████████████████████████| 51kB 4.9MB/s 
     |████████████████████████████████| 61kB 5.1MB/s 
     |████████████████████████████████| 61kB 5.7MB/s 
  Found existing installation: fastai 1.0.61
    Uninstalling fastai-1.0.61:
      Successfully uninstalled fastai-1.0.61


In [3]:
import pandas 
import fastbook
import voila

In [ ]:
from fastai.vision.all import *
from fastai.vision.widgets import *
from pandas import *
from fastbook import * 

# The Amazing Fish Classifier!

You need to know whether you're being chased by a dangerous grizzly, or a sweet teddy bear, and you need an answer *fast*? Then you've come to the right place. Take a pic of the potentially vicious killer, and click 'upload' to classify it. (Important: this only handles grizzly bears, black bears, and teddy bears. It will **not** give a sensible answer for polar bears, a bear market, a bear of a man, or hot dogs.

----

In [ ]:
path = Path()
learn_inf = load_learner(path/'export-res34-047.pkl', cpu=True)
btn_upload = widgets.FileUpload()
out_pl = widgets.Output()
out_pl_df = widgets.Output()
lbl_pred = widgets.Label()

/home/jhoward/anaconda3/lib/python3.7/site-packages/torch/serialization.py:649: SourceChangeWarning: source code of class 'torch.nn.modules.container.Sequential' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/home/jhoward/anaconda3/lib/python3.7/site-packages/torch/serialization.py:649: SourceChangeWarning: source code of class 'torch.nn.modules.conv.Conv2d' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/home/jhoward/anaconda3/lib/python3.7/site-packages/torch/serialization.py:649: SourceChangeWarning: source code of class 'torch.nn.modules.batchnorm.BatchNorm2d' has changed. you can retrieve the original source code by ac

UnpicklingError: invalid load key, '\x0a'.

In [ ]:
def create_df(): 
    fish = {'Familie': ['Leuciscidae', 'Icaluridae', 'Cobitidae', 'Cyprinidae', 'Esocidae', 'Percidae', 'Salmonidae'],
        'Spezies': ['abramis_brama', 'ameiurus_nebulosus', 'cobitis_taenia', 'cyprinus_carpio', 'esox_lucius', 'gymnocephalus_cernua', 'salmo_trutta'],
        'Name': ['Brasse', 'Katzenwels', 'Dorngrundel', 'Karpfen', 'Hecht', 'Kaulbarsch', 'Forelle'],
        'Koerperform':['Brasse', 'Katzenwels', 'Dorngrundel', 'Karpfen', 'Hecht', 'Kaulbarsch', 'Forelle'],
        'Mund': ['Brasse', 'Katzenwels', 'Dorngrundel', 'Karpfen', 'Hecht', 'Kaulbarsch', 'Forelle'],
        'Farbe': ['Brasse', 'Katzenwels', 'Dorngrundel', 'Karpfen', 'Hecht', 'Kaulbarsch', 'Forelle'], 
        'Rueckenflosse': ['Brasse', 'Katzenwels', 'Dorngrundel', 'Karpfen', 'Hecht', 'Kaulbarsch', 'Forelle'], 
        'Bauchflosse':['Brasse', 'Katzenwels', 'Dorngrundel', 'Karpfen', 'Hecht', 'Kaulbarsch', 'Forelle'], 
        'Schwanzflosse': ['Brasse', 'Katzenwels', 'Dorngrundel', 'Karpfen', 'Hecht', 'Kaulbarsch', 'Forelle'],
        'Schuppen': ['Brasse', 'Katzenwels', 'Dorngrundel', 'Karpfen', 'Hecht', 'Kaulbarsch', 'Forelle'],
        'Schonzeit': ['Brasse', 'Katzenwels', 'Dorngrundel', 'Karpfen', 'Hecht', 'Kaulbarsch', 'Forelle'],
        'Schonmaß': ['Brasse', 'Katzenwels', 'Dorngrundel', 'Karpfen', 'Hecht', 'Kaulbarsch', 'Forelle']
        }
    df = pandas.DataFrame(fish, columns = ['Familie', 'Spezies', 'Name', 'Koerperform', 'Mund', 'Farbe', 'Rueckenflosse', 'Bauchflosse','Schwanzflosse', 'Schuppen', 'Schonzeit','Schonmaß'], index=['abramis_brama', 'ameiurus_nebulosus', 'cobitis_taenia', 'cyprinus_carpio', 'esox_lucius', 'gymnocephalus_cernua', 'salmo_trutta'])
    return df 

In [ ]:
def on_data_change(change):
    lbl_pred.value = ''
    df = create_df()
    img = PILImage.create(btn_upload.data[-1])
    pred,pred_idx,probs = learn_inf.predict(img)
    out_pl.clear_output()
    with out_pl: display(img.to_thumb(128,128))
    out_pl_df.clear_output()
    with out_pl_df: display(df.loc[f'{pred}'])
    lbl_pred.value = f'Prediction: {pred}; Probability: {probs[pred_idx]:.04f}'

In [ ]:
btn_upload.observe(on_data_change, names=['data'])

In [ ]:
display(HBox([VBox([widgets.Label('Select your bear!'), btn_upload, out_pl, lbl_pred]), VBox([widgets.Label('Informationen'), out_pl_df])]))